In [3]:
from sympy.solvers import solve
from sympy import Symbol, log, exp
import sympy as sp

In [4]:
def get_prior(target_palloff, num_features=500):
    x = Symbol('x')
    eq = sp.Eq(num_features*log(1-x), log(target_palloff))
    sol = solve(eq, x)
    assert len(sol) == 1
    return sol[0]

In [5]:
prior = get_prior(0.4)

In [6]:
print(prior)

0.00183090331161104


In [108]:
import math
def sigmoid(x):
    return 1 / (1 + exp(-x))

def derive_params(prior_palloff_calibrate, target_palloff, label=True, num_features_observe=500, num_features_calibrate=500):
    # calculate prior prob of thetas
    prior = get_prior(prior_palloff_calibrate, num_features=num_features_calibrate)
    
    # calculate posterior prob of thetas based on target p_all_off_observes
    target_posterior = get_prior(target_palloff, num_features=num_features_observe)
    
    # calculate prior p_all_off_observe, used in updates
    prior_palloff_observe = math.pow((1-prior), num_features_observe)
#     prior_palloff_observe = math.pow(prior_palloff_calibrate, num_features_observe-num_features_calibrate)
#     print("prior p all off observe:", prior_palloff_observe)
#     print("prior:", prior)
#     print("posterior p all off observe:", target_palloff, (1-target_posterior)**num_features_observe)
        
    log_alpha = Symbol('log_alpha')
    # if label is True, subtract; else, add
    if label:
        term = -1*prior_palloff_observe*log_alpha
    else:
        term = prior_palloff_observe*log_alpha
        
#     print("term:", term)
#     print(log(sigmoid(log(prior)-log(1-prior)+term)))
#     print(log(target_posterior))
    
#     eq = sp.Eq(sigmoid(log(prior)-log(1-prior)+term), target_posterior)
    eq = sp.Eq(log(sigmoid(log(prior)-log(1-prior)+term)), log(target_posterior))
    sol = solve(eq, log_alpha)
    
#     assert len(sol) == 1, sol
    sol = sol[0]
    
    lla = log(sol)
    
    return prior, lla

In [109]:
# get hyperparams for positive updates (unif, eig_train_model)
num_features = 500 # our estimate for num features in a random sequence
for prior_palloff in [0.5, 0.3, 0.1]:
    print("")
    print(f"prior palloff: {prior_palloff}")
    for posterior_palloff in [0.4, 0.3, 0.2, 0.1, 0.05, 0.01]:
        if posterior_palloff >= prior_palloff:
            continue
        
        prior, lla = derive_params(prior_palloff, posterior_palloff, label=False, 
                                   num_features_observe=num_features,
                                   num_features_calibrate=500,
                                  )

        print(f'prior={prior}\tlla={lla}\t\t(posterior_palloff={posterior_palloff})')


prior palloff: 0.5
prior=0.00138533389897108	lla=-0.582269524436029		(posterior_palloff=0.4)
prior=0.00138533389897108	lla=0.100117997493526		(posterior_palloff=0.3)
prior=0.00138533389897108	lla=0.522731931474557		(posterior_palloff=0.2)
prior=0.00138533389897108	lla=0.877263465295860		(posterior_palloff=0.1)
prior=0.00138533389897108	lla=1.07568861873667		(posterior_palloff=0.05)
prior=0.00138533389897108	lla=1.33374133882702		(posterior_palloff=0.01)

prior palloff: 0.3
prior=0.00240504883318384	lla=-0.0316148863778716		(posterior_palloff=0.2)
prior=0.00240504883318384	lla=0.772427968403264		(posterior_palloff=0.1)
prior=0.00240504883318384	lla=1.11334210926109		(posterior_palloff=0.05)
prior=0.00240504883318384	lla=1.50033511229104		(posterior_palloff=0.01)

prior palloff: 0.1
prior=0.00459458264847305	lla=0.970210628375159		(posterior_palloff=0.05)
prior=0.00459458264847305	lla=1.93939240693089		(posterior_palloff=0.01)


In [110]:
import pandas as pd
words_with_feats_eng = pd.read_csv('words_with_feats_english.csv')
avg_num_feats = words_with_feats_eng['NumFeatures'].mean()
print(f'avg # feats: {avg_num_feats}')

avg # feats: 1797.6219567177639


In [ ]:
# get hyperparams for positive updates (train)
num_features = 1798 # our estimate for num features in a train sequence
for prior_palloff in [0.5, 0.3, 0.1]:
    print("")
    print(f"prior palloff: {prior_palloff}")
    for posterior_palloff in [0.8, 0.9, 0.95, 0.99]:
        if posterior_palloff <= prior_palloff:
            continue
        
        prior, lla = derive_params(prior_palloff, posterior_palloff, label=True, 
                                   num_features_observe=num_features,
                                   num_features_calibrate=500,
                                  )

        print(f'prior={prior}\tlla={lla}\t\t(posterior_palloff={posterior_palloff})')


prior palloff: 0.5
prior=0.00138533389897108	lla=3.37379282024428		(posterior_palloff=0.8)
prior=0.00138533389897108	lla=3.64450190689220		(posterior_palloff=0.9)
prior=0.00138533389897108	lla=3.84947003210775		(posterior_palloff=0.95)
prior=0.00138533389897108	lla=4.19987456939477		(posterior_palloff=0.99)

prior palloff: 0.3
prior=0.00240504883318384	lla=5.41687946870128		(posterior_palloff=0.8)
prior=0.00240504883318384	lla=5.64240070601870		(posterior_palloff=0.9)
prior=0.00240504883318384	lla=5.81942728026585		(posterior_palloff=0.95)
prior=0.00240504883318384	lla=6.13231740872503		(posterior_palloff=0.99)

prior palloff: 0.1
